In [ ]:
import string
chars_punctuation = string.punctuation
chars_whitespace = string.whitespace
chars_alphanumeric = string.ascii_letters + string.digits


def convert_to_ascii(utf8_string):
    try:
        return utf8_string.encode('ascii', 'ignore').decode('ascii')
    except UnicodeEncodeError as e:
        print(f"Conversion from UTF-8 to ASCII failed. Error: {e}")
        print(utf8_string)

In [31]:
# Convert the files in the data directory to ASCII
import os

filedir_utf8 = "data_utf8"
filedir_ascii = "data_ascii"
# Load all the files in the directory
files = os.listdir(filedir_utf8)

for file in files:
    with open(filedir_utf8 + "/" + file, "r", encoding="utf-8") as f:
        with open(filedir_ascii + "/" + file, "w", encoding="ascii") as f_new:
            for line in f:
                # line = normalizer.normalize_str(line)
                f_new.write(convert_to_ascii(line))

In [34]:
# Replace all punctuation with a space
# Convert all types of whitespace to a single space
import re

filedir_ascii = "data_ascii"
filedir_procd = "data_procd"


for file in files:
    with open(filedir_ascii + "/" + file, "r", encoding="ascii") as f:
        with open(filedir_procd + "/" + file, "w", encoding="ascii") as f_new:
            for line in f:
                for char in chars_punctuation:
                    line = line.replace(char, " ")
                for char in chars_whitespace:
                    line = line.replace(char, " ")
                
                # Shrink multiple spaces to a single space
                line = re.sub(' +', ' ', line)

                f_new.write(line)

In [44]:
from collections import Counter
# Convert in to a list of words, for all lines in the file
filedir_procd = "data_procd"

# for file in files:
word_counter = Counter()
for file in files:
    print(f"Processing file: {file}")
    with open(filedir_procd + "/" + file, "r", encoding="ascii") as f:
        for line in f:
            word_counter.update(line.split())
word_counts_dict = dict(word_counter)

import json

word_counts_dict = dict(word_counter.most_common())
output_json_path = 'word_counts.json'
with open(output_json_path, 'w', encoding='ascii') as json_file:
    json.dump(word_counts_dict, json_file, indent=4)

Processing file: wikipedia_1M_0.txt
Processing file: wikipedia_1M_1.txt
Processing file: wikipedia_1M_2.txt
Processing file: wikipedia_1M_3.txt
Processing file: wikipedia_1M_4.txt
Processing file: wikipedia_1M_5.txt
Processing file: wikipedia_1M_6.txt
Processing file: wikipedia_1M_7.txt
Processing file: wikipedia_1M_8.txt
Processing file: wikipedia_1M_9.txt


In [ ]:
sorted_word_counter = word_counter.most_common()
total_words = sum(word_counter.values())
cumulative_percentage = 0
selected_words = []
for word, count in sorted_word_counter:
    word_percentage = count / total_words
    cumulative_percentage += word_percentage
    selected_words.append((word, count))
    
    if cumulative_percentage >= 0.90:
        break

print(f"Total words: {total_words}")
print(f"Selected words: {len(selected_words)}")

output_selected_json_path = 'word_counts_90.json'
with open('word_counts_90.json', 'w', encoding='ascii') as json_file:
    json.dump(selected_words, json_file, indent=4)

In [4]:
import json
# Load the selected words
with open('word_counts_90.json', 'r', encoding='ascii') as json_file:
    selected_words = json.load(json_file)

In [5]:
words_dict = dict(selected_words)
words_dict = {key + ' ': value for key, value in words_dict.items()}
words_lst = list(words_dict.keys())

bytepairs = []
for w in words_lst:
    word_bps = []
    for c in w:
        word_bps.append(c)
    bytepairs.append(word_bps)

bp_filename = 'bytepairs.txt'
with open(bp_filename, 'w', encoding='ascii') as f:
    pass
    
output_bps = []
output_buffer = []
top_pair = None

for n in range(10000):
    counts = dict()
    for j in range(len(bytepairs)):
        word = bytepairs[j]
        original_word = word
        i=0
        while i < len(word) - 1:
            pair = (word[i], word[i+1])
            
            # If we have a top pair, we need to stick it together the rebulid the word
            if pair == top_pair:
                new_bytepairs = []
                k=0
                while k < len(word):
                    if k == len(word) - 1:
                        new_bytepairs.append(word[k])

                    elif (word[k], word[k+1]) == top_pair:
                        new_bytepairs.append(word[k] + word[k+1])
                        k += 1
                        
                    else:
                        new_bytepairs.append(word[k])
                    k += 1

                word = new_bytepairs
                bytepairs[j] = word # Replace the word char pairing with the new pairing


                # Increment the prev word index if we have stuck together a pair

                # Skip to the next word if this is now the end of the word
                # I.e. if len is now equal to i, given that we have incremented i and the word is now shorter
                if i == len(word) -1:
                    continue
                pair = (word[i], word[i+1])
                i += 1

            if pair in counts:
                counts[pair] += words_dict[''.join(word)]
            else:
                counts[pair] = words_dict[''.join(word)]

            i+=1

    # Sort the counts
    counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)

    # Extract and append the top pair
    top_pair = counts[0][0]
    output_bps.append(top_pair[0] + top_pair[1])
    output_buffer.append(top_pair[0] + top_pair[1])
    if n % 100 == 99:
        print(f"{n+1}th Top pair: {top_pair}, count: {counts[0][1]}")
        # Append the output buffer to file
        with open(bp_filename, 'a', encoding='ascii') as f:
            for bp in output_buffer:
                f.write(bp + '\n')
        output_buffer = []

99th Top pair: ('h', 'e '), count: 581895
199th Top pair: ('a', 'b'), count: 271495
299th Top pair: ('em', 'ber '), count: 166778
399th Top pair: ('b', 'ec'), count: 124395
499th Top pair: ('be', 'tw'), count: 93922
599th Top pair: ('m', 'em'), count: 78413
699th Top pair: ('devel', 'op'), count: 65873
799th Top pair: ('albu', 'm '), count: 55446
899th Top pair: ('le', 'ague '), count: 47723
999th Top pair: ('v', 'ir'), count: 41925
1099th Top pair: ('n', 'see '), count: 38554
1199th Top pair: ('200', '8 '), count: 34466
1299th Top pair: ('g', 'l'), count: 31241
1399th Top pair: ('gh', 'ts '), count: 28737
1499th Top pair: ('to', 'tal '), count: 26477
1599th Top pair: ('c', 'ase '), count: 24260
1699th Top pair: ('ous', 'ly '), count: 22282
1799th Top pair: ('201', '7 '), count: 20895
1899th Top pair: ('d', 'augh'), count: 19612
1999th Top pair: ('gr', 'and '), count: 18245
2099th Top pair: ('h', 'ill '), count: 17050
2199th Top pair: ('ma', 'y'), count: 16103
2299th Top pair: ('t', 't